In [1]:
import numpy as np
import pandas as pd

# read all single-csv files into dataframes
games_df = pd.read_csv("games.csv")
player_play_df = pd.read_csv("player_play.csv")
players_df = pd.read_csv("players.csv")
plays_df = pd.read_csv("plays.csv")


In [2]:
plays_cleaned = plays_df[['gameId', 'playId', 'down', 'yardsToGo', 'yardsGained', 'quarter', 'gameClock', 'offenseFormation', 'receiverAlignment', 'isDropback', 'passResult', 'passLength', 'playAction', 'timeToThrow', 'timeInTackleBox', 'timeToSack', 'unblockedPressure', 'qbKneel', 'penaltyYards', 'pff_passCoverage', 'pff_manZone']]
plays_cleaned.head()

,gameId,playId,down,yardsToGo,yardsGained,quarter,gameClock,offenseFormation,receiverAlignment,isDropback,...,passLength,playAction,timeToThrow,timeInTackleBox,timeToSack,unblockedPressure,qbKneel,penaltyYards,pff_passCoverage,pff_manZone
0,2022102302,2655,1,10,9,3,01:54,EMPTY,3x2,True,...,6.0,False,2.990,2.990,NaN,False,0,NaN,Cover-3,Zone
1,2022091809,3698,1,10,4,4,02:13,EMPTY,3x2,True,...,4.0,False,1.836,1.836,NaN,False,0,NaN,Quarters,Zone
2,2022103004,3146,3,12,6,4,02:00,SHOTGUN,2x2,True,...,-4.0,False,2.236,2.236,NaN,False,0,NaN,Quarters,Zone
3,2022110610,348,2,10,4,1,09:28,SHOTGUN,2x2,True,...,-6.0,False,2.202,2.202,NaN,False,0,NaN,Quarters,Zone
4,2022102700,2799,2,8,-1,3,02:16,PISTOL,3x1,False,...,NaN,True,NaN,NaN,NaN,NaN,0,NaN,Cover-1,Man


In [5]:
#insert success rate into plays_cleaned
success = []
for down, distance in plays_cleaned.iterrows():
    if distance['down'] == 1 and distance['yardsGained'] / distance['yardsToGo'] >= 0.4:
        success.append(True)
    elif distance['down'] == 2 and distance['yardsGained'] / distance['yardsToGo'] >= 0.6:
        success.append(True)
    elif distance['down'] == 3 and distance['yardsGained'] / distance['yardsToGo'] >= 1:
        success.append(True)
    elif distance['down'] == 4 and distance['yardsGained'] / distance['yardsToGo'] >= 1:
        success.append(True)
    else:
        success.append(False)
plays_cleaned.loc[:, 'success'] = success


        
        




C:\Users\stsch\AppData\Local\Temp\ipykernel_25580\337348853.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plays_cleaned.loc[:, 'success'] = success


In [7]:
#select columns from player plays

player_play_cleaned = player_play_df[['gameId', 'playId', 'nflId', 'teamAbbr', 'hadRushAttempt', 'rushingYards', 'hadPassReception', 'receivingYards', 'yardageGainedAfterTheCatch', 'fumbles', 'fumbleLost', 'quarterbackHit', 'tackleForALoss', 'tackleForALossYardage', 'causedPressure', 'timeToPressureAsPassRusher', 'getOffTimeAsPassRusher', 'inMotionAtBallSnap', 'motionSinceLineset']]
player_play_cleaned.head()

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadPassReception,receivingYards,yardageGainedAfterTheCatch,fumbles,fumbleLost,quarterbackHit,tackleForALoss,tackleForALossYardage,causedPressure,timeToPressureAsPassRusher,getOffTimeAsPassRusher,inMotionAtBallSnap,motionSinceLineset
0,2022090800,56,35472,BUF,0,0,0,0,0,0,0,0,0,0,False,NaN,NaN,False,False
1,2022090800,56,42392,BUF,0,0,0,0,0,0,0,0,0,0,False,NaN,NaN,False,False
2,2022090800,56,42489,BUF,0,0,1,6,1,0,0,0,0,0,False,NaN,NaN,False,True
3,2022090800,56,44875,BUF,0,0,0,0,0,0,0,0,0,0,False,NaN,NaN,False,False
4,2022090800,56,44985,BUF,0,0,0,0,0,0,0,0,0,0,False,NaN,NaN,False,False


In [9]:
#clean player plays data, isolate only plays with a player in motion

player_plays_with_motion = player_play_cleaned.mask(player_play_cleaned['motionSinceLineset'].eq(False))   #isolate the players that go into motion pre-snap
player_plays_with_motion = player_plays_with_motion.dropna(subset = ['motionSinceLineset'])                #remove NA values (defensive players, etc)
player_plays_with_motion = player_plays_with_motion[['gameId', 'playId', 'motionSinceLineset']].drop_duplicates()                #drop duplicates (some plays have multiple players in motion)




        

In [11]:
#compare total unique plays vs number of plays with pre-snap motion
motion_percentage = len(player_plays_with_motion) / len(plays_cleaned)
print(motion_percentage)

#league-wide, 30.6% of plays had pre-snap motion

0.3062515504837509


In [13]:
# add motion true/false to plays_cleaned

ppwm_for_merge = player_plays_with_motion[['gameId', 'playId', 'motionSinceLineset']]

plays_cleaned_with_motion = pd.merge(plays_cleaned, ppwm_for_merge, how='left')



In [15]:
plays_cleaned_with_motion.head(100)

,gameId,playId,down,yardsToGo,yardsGained,quarter,gameClock,offenseFormation,receiverAlignment,isDropback,...,timeToThrow,timeInTackleBox,timeToSack,unblockedPressure,qbKneel,penaltyYards,pff_passCoverage,pff_manZone,success,motionSinceLineset
0,2022102302,2655,1,10,9,3,01:54,EMPTY,3x2,True,...,2.990,2.990,NaN,False,0,NaN,Cover-3,Zone,True,NaN
1,2022091809,3698,1,10,4,4,02:13,EMPTY,3x2,True,...,1.836,1.836,NaN,False,0,NaN,Quarters,Zone,True,NaN
2,2022103004,3146,3,12,6,4,02:00,SHOTGUN,2x2,True,...,2.236,2.236,NaN,False,0,NaN,Quarters,Zone,False,NaN
3,2022110610,348,2,10,4,1,09:28,SHOTGUN,2x2,True,...,2.202,2.202,NaN,False,0,NaN,Quarters,Zone,False,NaN
4,2022102700,2799,2,8,-1,3,02:16,PISTOL,3x1,False,...,NaN,NaN,NaN,NaN,0,NaN,Cover-1,Man,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022100208,1348,2,6,8,2,05:24,SINGLEBACK,3x1,True,...,NaN,NaN,NaN,False,0,NaN,Cover-1,Man,True,NaN
96,2022100600,2300,1,10,0,3,14:21,SHOTGUN,2x2,True,...,1.968,1.968,NaN,False,0,NaN,Cover-3 Seam,Zone,False,NaN
97,2022091811,120,1,10,6,1,13:00,I_FORM,2x1,False,...,NaN,NaN,NaN,NaN,0,NaN,Cover-2,Zone,True,NaN
98,2022103005,79,1,5,3,1,15:00,SINGLEBACK,2x2,False,...,NaN,NaN,NaN,NaN,0,NaN,Quarters,Zone,True,NaN


In [67]:
#success rate for plays with motion vs those without
success_motion = 0
fail_motion = 0
success_no_motion = 0
fail_no_motion = 0
NA_count = 0
for success, motion in plays_cleaned_with_motion.iterrows():
    if motion['success'] == True and motion['motionSinceLineset'] == True:
        success_motion += 1
    elif motion['success'] == False and motion['motionSinceLineset'] == True:
        fail_motion += 1
    elif motion['success'] == True and motion['motionSinceLineset'] != True:
        success_no_motion += 1
    elif motion['success'] == False and motion['motionSinceLineset'] != True:
        fail_no_motion +=1
    else:
        NA_count +=1 

print(success_motion)
print(fail_motion)
print(success_no_motion)
print(fail_no_motion)
print(NA_count)

motion_success = success_motion / (success_motion + fail_motion)
no_motion_success = success_no_motion / (success_no_motion + fail_no_motion)
print(motion_success)
print(no_motion_success)
    

2491
2447
5196
5990
0
0.5044552450384772
0.46450920793849454


In [71]:
#success rate for motion/no motion by pass/run 

pass_success_motion = 0
pass_fail_motion = 0
pass_success_no_motion = 0
pass_fail_no_motion = 0
run_success_motion = 0
run_fail_motion = 0
run_success_no_motion = 0
run_fail_no_motion = 0
NA_count = 0

for success, motion in plays_cleaned_with_motion.iterrows():
    if motion['isDropback'] == True:
        if motion['success'] == True and motion['motionSinceLineset'] == True:
            pass_success_motion += 1
        elif motion['success'] == False and motion['motionSinceLineset'] == True:
            pass_fail_motion += 1
        elif motion['success'] == True and motion['motionSinceLineset'] != True:
            pass_success_no_motion += 1
        elif motion['success'] == False and motion['motionSinceLineset'] != True:
            pass_fail_no_motion += 1
        else:
            NA_count += 1
    else:
        if motion['success'] == True and motion['motionSinceLineset'] == True:
            run_success_motion += 1
        elif motion['success'] == False and motion['motionSinceLineset'] == True:
            run_fail_motion += 1 
        elif motion['success'] == True and motion['motionSinceLineset'] != True:
            run_success_no_motion += 1
        elif motion['success'] == False and motion['motionSinceLineset'] != True:
            run_fail_no_motion += 1    
        else:
            NA_count +=1 

pass_motion_success = pass_success_motion / (pass_success_motion + pass_fail_motion)
pass_no_motion_success = pass_success_no_motion / (pass_success_no_motion + pass_fail_no_motion)
run_motion_success = run_success_motion / (run_success_motion + run_fail_motion)
run_no_motion_success = run_success_no_motion / (run_success_no_motion + run_fail_no_motion)
print(pass_motion_success)
print(pass_no_motion_success)
print(run_motion_success)
print(run_no_motion_success)
print(NA_count)

0.5057283142389526
0.442668863261944
0.5007861635220126
0.4904222048475371
0
9736


In [ ]:
#attempting to group by formation and receiver alignment

#df = plays_cleaned_with_motion.groupby(['success', 'motionSinceLineset', 'isDropback', 'offenseFormation', 'receiverAlignment']).size()

#df.head(50)

#df.to_excel("success_by_formation_and_alignment.xlsx")

In [ ]:
#looking at time to pressure with motion vs no motion


player_play_with_pressure = player_play_cleaned[['gameId', 'playId', 'timeToPressureAsPassRusher']].dropna()  #isolating players with pass rush pressure

motion_plays_pressure_time = pd.merge(player_play_with_pressure, ppwm_for_merge, how='left')  #merging players with pass rush pressure with our plays with motion
motion_plays_pressure_time.sort_values(by=['gameId', 'playId', 'timeToPressureAsPassRusher'])  #since multiple players can achieve pass rush pressure on a single play,
motion_plays_pressure_time.drop_duplicates(subset=['gameId', 'playId'], inplace=True)   #we sort by quickest time per play and drop the rest
motion_plays_pressure_time['motionSinceLineset'].fillna(False, inplace=True)  #we fill in the NaN values for motion so we can do boolean operations
motion_pressure_times = motion_plays_pressure_time.loc[motion_plays_pressure_time['motionSinceLineset']==True]  #here we isolate the pressure times for plays with motion
no_motion_pressure_times = motion_plays_pressure_time.loc[motion_plays_pressure_time['motionSinceLineset']==False]  #and #here we isolate the pressure times for plays without motion        
motion_pressure_times.head(50)

average_pressure_time_motion = motion_pressure_times['timeToPressureAsPassRusher'].mean()  #we take the means for each group
average_pressure_time_no_motion = no_motion_pressure_times['timeToPressureAsPassRusher'].mean()

print(average_pressure_time_motion)
print(average_pressure_time_no_motion)

#The average time to pressure on plays with motion is 2.72 seconds
#The average time to pressure on plays without motion is 2.78 seconds

In [119]:
#looking at pass pressure rate motion vs no motion

player_caused_pressure = player_play_cleaned[['gameId', 'playId', 'causedPressure']].dropna() #isolating players with pass rush pressure

plays_with_pressure = pd.merge(player_caused_pressure, plays_cleaned_with_motion, how='left') #we merge the causedPressure column into the plays cleaned with motion dataframe
plays_with_pressure = plays_with_pressure.loc[plays_with_pressure['isDropback']==True] #we isolate pass plays
plays_with_pressure = plays_with_pressure.loc[plays_with_pressure['causedPressure']==True] #we isolate plays with pass rush pressure
plays_with_pressure.sort_values(by=['gameId', 'playId', 'causedPressure'])  #since multiple players can achieve pass rush pressure on a single play,
plays_with_pressure.drop_duplicates(subset=['gameId', 'playId'], inplace=True)   #we drop duplicates so only one pressure is counted for a particular play
plays_with_pressure['motionSinceLineset'].fillna(False, inplace=True) #we fill in the NaN values for motion so we can do boolean operations
motion_plays_with_pressure = plays_with_pressure.loc[plays_with_pressure['motionSinceLineset']==True] #we isolate the pass pressure plays that had pre-snap motion
motion_plays_with_pressure = motion_plays_with_pressure['gameId'].count()


no_motion_plays_with_pressure = plays_with_pressure.loc[plays_with_pressure['motionSinceLineset']==False] #we isolate the pass pressure plays that did not have pre-snap motion
no_motion_plays_with_pressure = no_motion_plays_with_pressure['gameId'].count()

#we repeat the last few steps to isolate plays without pass pressure
plays_without_pressure = pd.merge(player_caused_pressure, plays_cleaned_with_motion, how='left') #we merge the causedPressure column into the plays cleaned with motion dataframe
plays_without_pressure = plays_without_pressure.loc[plays_without_pressure['isDropback']==True] #we isolate pass plays
plays_without_pressure = plays_without_pressure.loc[plays_without_pressure['causedPressure']==False] #we isolate plays without pass rush pressure
plays_without_pressure.sort_values(by=['gameId', 'playId', 'causedPressure'])  #since multiple players can achieve pass rush pressure on a single play,
plays_without_pressure.drop_duplicates(subset=['gameId', 'playId'], inplace=True)   #we drop duplicates so only one pressure is counted for a particular play
plays_without_pressure['motionSinceLineset'].fillna(False, inplace=True) #we fill in the NaN values for motion so we can do boolean operations
motion_plays_without_pressure = plays_without_pressure.loc[plays_without_pressure['motionSinceLineset']==True] #we isolate the pass pressure plays that had pre-snap motion
motion_plays_without_pressure = motion_plays_without_pressure['gameId'].count()


no_motion_plays_without_pressure = plays_without_pressure.loc[plays_without_pressure['motionSinceLineset']==False] #we isolate the pass pressure plays that did not have pre-snap motion
no_motion_plays_without_pressure = no_motion_plays_without_pressure['gameId'].count()




motion_pressure_rate = (motion_plays_with_pressure / (motion_plays_with_pressure + motion_plays_without_pressure)) * 100 #converting decimal to percent
no_motion_pressure_rate = (no_motion_plays_with_pressure / (no_motion_plays_with_pressure + no_motion_plays_without_pressure)) * 100 #converting decimal to percent

motion_pressure_rate = round(motion_pressure_rate, 2) #rounding percentage to 2 decimal places
no_motion_pressure_rate = round(no_motion_pressure_rate, 2) #rounding percentage to 2 decimal places

print("The pressure rate on pass plays with pre-snap motion is {}.".format (motion_pressure_rate))
print("The pressure rate on pass plays without pre-snap motion is {}.".format (no_motion_pressure_rate))

The pressure rate on pass plays with pre-snap motion is 23.03.
The pressure rate on pass plays without pre-snap motion is 26.32.


C:\Users\stsch\AppData\Local\Temp\ipykernel_25580\1833150406.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  plays_with_pressure['motionSinceLineset'].fillna(False, inplace=True) #we fill in the NaN values for motion so we can do boolean operations
C:\Users\stsch\AppData\Local\Temp\ipykernel_25580\1833150406.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set 

In [ ]:
#looking at play action success rate with motion
play_action_with_motion = plays_cleaned_with_motion.groupby(['motionSinceLineset', 'success', 'playAction', 'isDropback']).size()

print(play_action_with_motion)

#play action success raw numbers:
#success-play action-pass: 684
#success-play action-run: 31
#success-not play action-pass: 1170
#sucecss-not play action-run: 606
#fail-play action-pass: 537
#fail-play action-run: 24
#fail-not play action-pass: 1275
#fail-not play action-run: 611

#play action pass success rate: 61.0%
#play action run success rate: 56.4% (small sample size)
#no play action pass success rate: 52.1%
#no play action run success rate: 49.7%


In [ ]:
#tracking_week_1 = pd.read_csv("tracking_week_1.csv")
#tracking_week_1_df = tracking_week_1[['gameId', 'playId', 'nflId']]
#tracking_week_1.head()
#tracking_week_2 = pd.read_csv("tracking_week_2.csv", skiprows=1)
#tracking_week_3 = pd.read_csv("tracking_week_3.csv", skiprows=1)
#tracking_week_4 = pd.read_csv("tracking_week_4.csv", skiprows=1)
#tracking_week_5 = pd.read_csv("tracking_week_5.csv", skiprows=1)
#tracking_week_6 = pd.read_csv("tracking_week_6.csv", skiprows=1)
#tracking_week_7 = pd.read_csv("tracking_week_7.csv", skiprows=1)
#tracking_week_8 = pd.read_csv("tracking_week_8.csv", skiprows=1)
#tracking_week_9 = pd.read_csv("tracking_week_9.csv", skiprows=1)

In [ ]:
tracking_frames = [tracking_week_1_df, tracking_week_2_df, tracking_week_3_df, tracking_week_4_df, tracking_week_5_df, tracking_week_6_df, tracking_week_7_df, tracking_week_8_df, tracking_week_9_df]
tracking_all_weeks_df = pd.concat(tracking_frames)
tracking_all_weeks_df.shape